In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from PIL import Image
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten,Dense,Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = "TRAIN"
test_dir = "TEST"

datagen_train = ImageDataGenerator(
    validation_split = 0.2,
    width_shift_range=0.1,
    rotation_range=20,
    rescale = 1./255,
    horizontal_flip=True,
    zoom_range = 0.1
)

datagen_test = ImageDataGenerator(
    validation_split = 0.2,
    rescale = 1./255
)

train_gen = datagen_train.flow_from_directory(
    directory = train_dir,
    target_size = (224, 224),
    batch_size = 16,
    class_mode = "categorical",
    subset = "training"
)

test_gen = datagen_test.flow_from_directory(
    directory = test_dir,
    target_size = (224, 224),
    batch_size = 16,
    class_mode = "categorical",
)

model = Sequential([
    Conv2D(64, (3, 3), activation="relu", input_shape=(224, 224, 3), padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(96, (3, 3), activation="relu", padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu", padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(256, activation="relu"),
    Dropout(0.4),
    Dense(train_gen.num_classes, activation="softmax")
])



In [ ]:
optimizer = Adam()
model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy'])
epochs = 40  
batch_size = 16

model.summary()

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
history = model.fit(train_gen, epochs = epochs,validation_data = test_gen)

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
train_loss, train_acc = model.evaluate(train_gen)
test_loss, test_acc   = model.evaluate(test_gen)

print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_acc*100, test_acc*100))